In [1]:
import numpy as np
from datetime import datetime
from datetime import timedelta
import sys
import re
import os
import itertools
import glob
import pandas as pd

C:\Users\Dr Hassan.DESKTOP-HJCCLKQ\AppData\Local\Temp\ipykernel_19036\1990547235.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

def prep_data(feat_filename,lab_cls,target_length):
  wholeList = []
  X = pd.read_csv(feat_filename,delimiter=";")
  if len(X.columns)==4:
        X.columns = ['Timestamp', 'X', 'Y', 'Z']
  for t in range(0,len(X),target_length):
    new_list1=[]
    new_list2=[]
    new_list3=[]

    new_list1.extend(X['X'][t:t+target_length].tolist())
    new_list2.extend(X['Y'][t:t+target_length].tolist())
    new_list3.extend(X['Z'][t:t+target_length].tolist())
    wholeList.append(new_list1 + new_list2 + new_list3)
  wholeList.pop() #removes the last element of the list in our case Last List
  print(len(wholeList))
  Y = [lab_cls for _ in range(len(wholeList))]
  return wholeList, Y


In [12]:
import pandas as pd

def prep_data(feat_filename, lab_cls, target_length):
    data_list = []
    X = pd.read_csv(feat_filename, delimiter=";")

    # Rename all columns to 'X', 'Y', 'Z'
    X.columns = ['Timestamp','X', 'Y', 'Z']

    for t in range(0, len(X), target_length):
        data_point = {
            'X': X['X'][t:t+target_length].tolist(),
            'Y': X['Y'][t:t+target_length].tolist(),
            'Z': X['Z'][t:t+target_length].tolist()
        }
        data_list.append(data_point)
    data_list.pop() #removes the last element of the list in our case Last List
    print(len(data_list))
    Y = [lab_cls for _ in range(len(data_list))]
    return data_list, Y


In [3]:
cls=0
def load_data(X,target_length):
    global cls
    print(X)
    if X.find('brushing_teeth')!=-1:
      cls=0

    elif X.find('cleaning_room')!=-1:
      cls=1

    elif X.find('handling_medications')!=-1:
      cls=2

    elif X.find('preparing_food')!=-1:
      cls=3

    elif X.find('styling_hair')!=-1:
      cls=4

    elif X.find('using_telephone')!=-1:
      cls=5

    elif X.find('washing_hands')!=-1:
      cls=6


    return prep_data(X,cls,target_length)

In [8]:
#training testing done separately to save time and memory
Y_train = []
JinsSensor_train = []
JinsSensorgyro_train = []
and_accel_train = []
and_gyro_train = []
PhoneAccelerometer_train = []
PhoneGravity_train = []
PhoneGyroscope_train = []
PhoneLinearAccelerometer_train = []
PhoneMagnetometer_train = []

min_length = float('inf')

subjects = glob.glob(r'C:\Users\Dr Hassan.DESKTOP-HJCCLKQ\Documents\sensory_data_Sync/*')
for sub in subjects:
    a = glob.glob(sub + '/*')
    for t in a:
        if 'Testing' in t or 'testing' in t:
            b = glob.glob(t + '/*')
            for u in b:
                print(u)
                c = glob.glob(u + '/*')
                for v in c:
                    d = glob.glob(v + '/*.csv')
                    print(d)
                    data_lengths = {}
                    if(v + '/JinsSensorgyro.csv'):
                      print('hello*********************************')
                      M4, Y = load_data(v + '/JinsSensorgyro.csv', 100)
                      JinsSensorgyro_train.extend(M4)
                      data_lengths['JinsSensorgyro'] = len(JinsSensorgyro_train)

                    for w in d:
                        if w.find('android.sensor.accelerometer') != -1:
                            L1, Y = load_data(w, 500)
                            and_accel_train.extend(L1)
                            data_lengths['and_accel'] = len(and_accel_train)

                        elif w.find('android.sensor.gyroscope') != -1:
                            L2, Y = load_data(w, 500)
                            and_gyro_train.extend(L2)
                            data_lengths['and_gyro'] = len(and_gyro_train)

                        elif w.find('JinsSensor') != -1:
                            L3, Y = load_data(w, 100)
                            JinsSensor_train.extend(L3)
                            data_lengths['JinsSensor'] = len(JinsSensor_train)

                        elif w.find('PhoneMagnetometer') != -1:
                            L9, Y = load_data(w, 500)
                            PhoneMagnetometer_train.extend(L9)
                            data_lengths['PhoneMagnetometer'] = len(PhoneMagnetometer_train)

                        elif w.find('PhoneAccelerometer') != -1:
                            L5, Y = load_data(w, 1000)
                            PhoneAccelerometer_train.extend(L5)
                            data_lengths['PhoneAccelerometer'] = len(PhoneAccelerometer_train)

                        elif w.find('PhoneGravity') != -1:
                            L6, Y = load_data(w, 1000)
                            PhoneGravity_train.extend(L6)
                            data_lengths['PhoneGravity'] = len(PhoneGravity_train)

                        elif w.find('PhoneGyroscope') != -1:
                            L7, Y = load_data(w, 1000)
                            PhoneGyroscope_train.extend(L7)
                            data_lengths['PhoneGyroscope'] = len(PhoneGyroscope_train)

                        elif w.find('PhoneLinearAccelerometer') != -1:
                            L8, Y = load_data(w, 1000)
                            PhoneLinearAccelerometer_train.extend(L8)
                            data_lengths['PhoneLinearAccelerometer'] = len(PhoneLinearAccelerometer_train)


                        Y_train.extend(Y)

                    min_length = min(data_lengths.values())

                    and_accel_train = and_accel_train[:min_length]
                    and_gyro_train = and_gyro_train[:min_length]
                    JinsSensor_train = JinsSensor_train[:min_length]
                    JinsSensorgyro_train = JinsSensorgyro_train[:min_length]
                    PhoneAccelerometer_train = PhoneAccelerometer_train[:min_length]
                    PhoneGravity_train = PhoneGravity_train[:min_length]
                    PhoneGyroscope_train = PhoneGyroscope_train[:min_length]
                    PhoneLinearAccelerometer_train = PhoneLinearAccelerometer_train[:min_length]
                    PhoneMagnetometer_train = PhoneMagnetometer_train[:min_length]
                    Y_train = Y_train[:min_length]

        else:
            continue


C:\Users\Dr Hassan.DESKTOP-HJCCLKQ\Documents\sensory_data_Sync\Subj1\testing\lefthand
['C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\Documents\\sensory_data_Sync\\Subj1\\testing\\lefthand\\brushing_teeth_composite_activity2018_03_17_14_29_42\\android.sensor.accelerometer.csv', 'C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\Documents\\sensory_data_Sync\\Subj1\\testing\\lefthand\\brushing_teeth_composite_activity2018_03_17_14_29_42\\android.sensor.gyroscope.csv', 'C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\Documents\\sensory_data_Sync\\Subj1\\testing\\lefthand\\brushing_teeth_composite_activity2018_03_17_14_29_42\\JinsSensor.csv', 'C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\Documents\\sensory_data_Sync\\Subj1\\testing\\lefthand\\brushing_teeth_composite_activity2018_03_17_14_29_42\\JinsSensorgyro.csv', 'C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\Documents\\sensory_data_Sync\\Subj1\\testing\\lefthand\\brushing_teeth_composite_activity2018_03_17_14_29_42\\PhoneAccelerometer.csv', 'C:\\Users\\Dr Hassan.DESKTOP-HJCCLKQ\\

In [9]:
print('JinsSensor_train',len( JinsSensor_train), 'JinsSensorgyro_train', len(JinsSensorgyro_train), 'and_accel_train', len(and_accel_train), 'and_gyro_train', len(and_gyro_train), 'PhoneAccelerometer_train', len(PhoneAccelerometer_train), 'PhoneGravity_train', len(PhoneGravity_train), 'PhoneGyroscope_train', len(PhoneGyroscope_train), 'PhoneLinearAccelerometer_train' ,len(PhoneLinearAccelerometer_train),'PhoneMagnetometer_train', len(PhoneMagnetometer_train), 'Y_train',len(Y_train))

JinsSensor_train 9251 JinsSensorgyro_train 9251 and_accel_train 9251 and_gyro_train 9251 PhoneAccelerometer_train 9251 PhoneGravity_train 9251 PhoneGyroscope_train 9251 PhoneLinearAccelerometer_train 9251 PhoneMagnetometer_train 9251 Y_train 9251


In [10]:
u,c=np.unique(Y_train, return_counts=True)
print(u)
print(c)

[0 1 2 3 4 5 6]
[1877 2055  590 2070  869 1030  760]


In [11]:
#Save data json format
import json

data_dict = {'JinsSensor_train': JinsSensor_train, 'JinsSensorgyro_train': JinsSensorgyro_train, 'and_accel_train': and_accel_train, 'and_gyro_train': and_gyro_train, 'PhoneAccelerometer_train': PhoneAccelerometer_train, 'PhoneGravity_train': PhoneGravity_train, 'PhoneGyroscope_train': PhoneGyroscope_train, 'PhoneLinearAccelerometer_train': PhoneLinearAccelerometer_train,'PhoneMagnetometer_train': PhoneMagnetometer_train, 'Y_train': Y_train}

# Save data to a file in JSON format
with open('CompositeTestdata1D.json', 'w') as file:
    json.dump(data_dict, file)

In [ ]:
#Complete code
Y_train = []
JinsSensor_train = []
JinsSensorgyro_train = []
and_accel_train = []
and_gyro_train = []
PhoneAccelerometer_train = []
PhoneGravity_train = []
PhoneGyroscope_train = []
PhoneLinearAccelerometer_train = []
PhoneMagnetometer_train = []

Y_test = []
JinsSensor_test = []
JinsSensorgyro_test = []
and_accel_test = []
and_gyro_test = []
PhoneAccelerometer_test = []
PhoneGravity_test = []
PhoneGyroscope_test = []
PhoneLinearAccelerometer_test = []
PhoneMagnetometer_test = []

min_length = float('inf')

subjects = glob.glob('/content/drive/MyDrive/sensory_data_Sync/*')
for sub in subjects:
    a = glob.glob(sub + '/*')
    for t in a:
        if t.split('/')[-1] == 'training':
            b = glob.glob(t + '/*')
            for u in b:
                c = glob.glob(u + '/*')
                for v in c:
                    d = glob.glob(v + '/*.csv')
                    data_lengths = {}
                    for w in d:
                        if w.find('android.sensor.accelerometer') != -1:
                            L1, Y = load_data(w, 500)
                            and_accel_train.extend(L1)
                            data_lengths['and_accel'] = len(and_accel_train)

                        elif w.find('android.sensor.gyroscope') != -1:
                            L2, Y = load_data(w, 500)
                            and_gyro_train.extend(L2)
                            data_lengths['and_gyro'] = len(and_gyro_train)

                        elif w.find('JinsSensor') != -1:
                            L3, Y = load_data(w, 100)
                            JinsSensor_train.extend(L3)
                            data_lengths['JinsSensor'] = len(JinsSensor_train)

                        elif w.find('JinsSensorgyro') != -1:
                            L4, Y = load_data(w, 100)
                            JinsSensorgyro_train.extend(L4)
                            data_lengths['JinsSensorgyro'] = len(JinsSensorgyro_train)

                        elif w.find('PhoneAccelerometer') != -1:
                            L5, Y = load_data(w, 1000)
                            PhoneAccelerometer_train.extend(L5)
                            data_lengths['PhoneAccelerometer'] = len(PhoneAccelerometer_train)

                        elif w.find('PhoneGravity') != -1:
                            L6, Y = load_data(w, 1000)
                            PhoneGravity_train.extend(L6)
                            data_lengths['PhoneGravity'] = len(PhoneGravity_train)

                        elif w.find('PhoneGyroscope') != -1:
                            L7, Y = load_data(w, 1000)
                            PhoneGyroscope_train.extend(L7)
                            data_lengths['PhoneGyroscope'] = len(PhoneGyroscope_train)

                        elif w.find('PhoneLinearAccelerometer') != -1:
                            L8, Y = load_data(w, 1000)
                            PhoneLinearAccelerometer_train.extend(L8)
                            data_lengths['PhoneLinearAccelerometer'] = len(PhoneLinearAccelerometer_train)

                        else:
                            L9, Y = load_data(w, 500)
                            PhoneMagnetometer_train.extend(L9)
                            data_lengths['PhoneMagnetometer'] = len(PhoneMagnetometer_train)

                        Y_train.extend(Y)

                    min_length = min(data_lengths.values())

                    and_accel_train = and_accel_train[:min_length]
                    and_gyro_train = and_gyro_train[:min_length]
                    JinsSensor_train = JinsSensor_train[:min_length]
                    JinsSensorgyro_train = JinsSensorgyro_train[:min_length]
                    PhoneAccelerometer_train = PhoneAccelerometer_train[:min_length]
                    PhoneGravity_train = PhoneGravity_train[:min_length]
                    PhoneGyroscope_train = PhoneGyroscope_train[:min_length]
                    PhoneLinearAccelerometer_train = PhoneLinearAccelerometer_train[:min_length]
                    PhoneMagnetometer_train = PhoneMagnetometer_train[:min_length]
                    Y_train = Y_train[:min_length]

        else:
            b = glob.glob(t + '/*')
            for u in b:
                c = glob.glob(u + '/*')
                for v in c:
                    d = glob.glob(v + '/*.csv')
                    data_lengths1 = {}
                    for w in d:
                        if w.find('android.sensor.accelerometer') != -1:
                            L1, Y = load_data(w, 500)
                            and_accel_test.extend(L1)
                            data_lengths1['and_accel'] = len(and_accel_test)

                        elif w.find('android.sensor.gyroscope') != -1:
                            L2, Y = load_data(w, 500)
                            and_gyro_test.extend(L2)
                            data_lengths1['and_gyro'] = len(and_gyro_test)

                        elif w.find('JinsSensor') != -1:
                            L3, Y = load_data(w, 100)
                            JinsSensor_test.extend(L3)
                            data_lengths1['JinsSensor'] = len(JinsSensor_test)

                        elif w.find('JinsSensorgyro') != -1:
                            L4, Y = load_data(w, 100)
                            JinsSensorgyro_test.extend(L4)
                            data_lengths1['JinsSensorgyro'] = len(JinsSensorgyro_test)

                        elif w.find('PhoneAccelerometer') != -1:
                            L5, Y = load_data(w, 1000)
                            PhoneAccelerometer_test.extend(L5)
                            data_lengths1['PhoneAccelerometer'] = len(PhoneAccelerometer_test)

                        elif w.find('PhoneGravity') != -1:
                            L6, Y = load_data(w, 1000)
                            PhoneGravity_test.extend(L6)
                            data_lengths1['PhoneGravity'] = len(PhoneGravity_test)

                        elif w.find('PhoneGyroscope') != -1:
                            L7, Y = load_data(w, 1000)
                            PhoneGyroscope_test.extend(L7)
                            data_lengths1['PhoneGyroscope'] = len(PhoneGyroscope_test)

                        elif w.find('PhoneLinearAccelerometer') != -1:
                            L8, Y = load_data(w, 1000)
                            PhoneLinearAccelerometer_test.extend(L8)
                            data_lengths1['PhoneLinearAccelerometer'] = len(PhoneLinearAccelerometer_test)

                        else:
                            L9, Y = load_data(w, 500)
                            PhoneMagnetometer_test.extend(L9)
                            data_lengths1['PhoneMagnetometer'] = len(PhoneMagnetometer_test)

                        Y_test.extend(Y)
                    min_length = min(data_lengths1.values())

                    and_accel_test = and_accel_test[:min_length]
                    and_gyro_test = and_gyro_test[:min_length]
                    JinsSensor_test = JinsSensor_test[:min_length]
                    JinsSensorgyro_test = JinsSensorgyro_test[:min_length]
                    PhoneAccelerometer_test = PhoneAccelerometer_test[:min_length]
                    PhoneGravity_test = PhoneGravity_test[:min_length]
                    PhoneGyroscope_test = PhoneGyroscope_test[:min_length]
                    PhoneLinearAccelerometer_test = PhoneLinearAccelerometer_test[:min_length]
                    PhoneMagnetometer_test = PhoneMagnetometer_test[:min_length]
                    Y_test = Y_test[:min_length]


/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/PhoneAccelerometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/android.sensor.accelerometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/android.sensor.gyroscope.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/PhoneGyroscope.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/PhoneGravity.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/PhoneLinearAccelerometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_21_35/PhoneMagnetometer.cs

<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)
<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)


13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/android.sensor.gyroscope.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/android.sensor.accelerometer.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/PhoneGravity.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/PhoneGyroscope.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/PhoneLinearAccelerometer.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/PhoneMagnetometer.csv
13
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_20_22/JinsSensor.csv
13


<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)
<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)


12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_15_59/PhoneLinearAccelerometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_15_59/PhoneMagnetometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_15_59/JinsSensorgyro.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_15_59/JinsSensor.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_01_35/android.sensor.accelerometer.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_01_35/android.sensor.gyroscope.csv
12
/content/drive/MyDrive/sensory_data_Sync/Subj6/testing/lefthand/washing_hands_composite_activity2018_03_25_14_01_35/PhoneAccelerometer.c

...

11
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/android.sensor.gyroscope.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/android.sensor.accelerometer.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/PhoneLinearAccelerometer.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/PhoneGravity.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/PhoneGyroscope.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/PhoneAccelerometer.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_42_33/PhoneMagnetometer.csv
9

<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)


9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/android.sensor.accelerometer.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/PhoneAccelerometer.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/android.sensor.gyroscope.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/PhoneMagnetometer.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/PhoneLinearAccelerometer.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/PhoneGyroscope.csv
8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_43_50/PhoneGravity.csv
8


<ipython-input-3-5cfbb8bf5389>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  X.set_axis(['Timestamp', 'X', 'Y', 'Z'], axis='columns', inplace= True)


8
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_36_39/android.sensor.gyroscope.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_36_39/android.sensor.accelerometer.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_36_39/PhoneAccelerometer.csv
9
/content/drive/MyDrive/sensory_data_Sync/Subj5/testing/lefthand/washing_hands_composite_activity2018_03_25_11_36_39/PhoneGravity.csv


In [ ]:
print(len(JinsSensor_train))


4183


In [ ]:
import json

data_dict = {'JinsSensor_train': JinsSensor_train, 'JinsSensorgyro_train': JinsSensorgyro_train, 'and_accel_train': and_accel_train, 'and_gyro_train': and_gyro_train, 'PhoneAccelerometer_train': PhoneAccelerometer_train, 'PhoneGravity_train': PhoneGravity_train, 'PhoneGyroscope_train': PhoneGyroscope_train, 'PhoneLinearAccelerometer_train': PhoneLinearAccelerometer_train,'PhoneMagnetometer_train': PhoneMagnetometer_train, 'Y_train': Y_train}

# Save data to a file in JSON format
with open('/content/drive/MyDrive/newVersion/Train/traindata.json', 'w') as file:
    json.dump(data_dict, file)

In [ ]:
data_dict1 = {'JinsSensor_test': JinsSensor_test, 'JinsSensorgyro_test': JinsSensorgyro_test, 'and_accel_test': and_accel_test, 'and_gyro_test': and_gyro_test, 'PhoneAccelerometer_test': PhoneAccelerometer_test, 'PhoneGravity_test': PhoneGravity_test, 'PhoneGyroscope_test': PhoneGyroscope_test, 'PhoneLinearAccelerometer_test': PhoneLinearAccelerometer_test,'PhoneMagnetometer_test': PhoneMagnetometer_test, 'Y_test': Y_test}

# Save data to a file in JSON format
with open('/content/drive/MyDrive/newVersion/Test/testdata.json', 'w') as file:
    json.dump(data_dict1, file)